In [58]:
import pandas as pd
import numpy as np
import sklearn
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import keras
df_1 = pd.read_csv("ptbdb_normal.csv", header=None)
df_2 = pd.read_csv("ptbdb_abnormal.csv", header=None)
df = pd.concat([df_1, df_2])

df_train, df_test = train_test_split(df, test_size=0.2, random_state=1337, stratify=df[187])


Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

In [59]:
RNN_OUTPUT_UNITS = [64, 128]
def get_model():
    inp = Input(shape=(187, 1))
    current_layer = keras.layers.Masking(mask_value=0., input_shape=(187, 1), name='Masked')(inp)
    for i, size in enumerate(RNN_OUTPUT_UNITS):
        notLast = i + 1 < len(RNN_OUTPUT_UNITS)
        layer = keras.layers.LSTM(size, return_sequences=notLast, dropout=0.2, name = 'LSTM' + str(i+1))
        current_layer = keras.layers.Bidirectional(layer, name = 'BiRNN' + str(i+1))(current_layer)
    current_layer = keras.layers.Dense(128, name='Dense1', activation='relu')(current_layer)
    current_layer = keras.layers.Dense(64, name='Dense2', activation='relu')(current_layer)
    current_layer = keras.layers.Dense(16, name='Dense3', activation='relu')(current_layer)
    logits = keras.layers.Dense(2, name='Output', activation='softmax')(current_layer)
    model = models.Model(inputs=inp, outputs=logits)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model

model = get_model()
file_path = "baseline_bidir_lstm_ptb.h5"
model.load_weights(file_path)
pred_test_bidir_logits= model.predict(X_test)
pred_test_bidir= np.argmax(pred_test_bidir_logits, axis=-1)

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 187, 1)            0         
_________________________________________________________________
Masked (Masking)             (None, 187, 1)            0         
_________________________________________________________________
BiRNN1 (Bidirectional)       (None, 187, 128)          33792     
_________________________________________________________________
BiRNN2 (Bidirectional)       (None, 256)               263168    
_________________________________________________________________
Dense1 (Dense)               (None, 128)               32896     
_________________________________________________________________
Dense2 (Dense)               (None, 64)                8256      
_________________________________________________________________
Dense3 (Dense)               (None, 16)                104

In [60]:
Y_test_sparse=[]
for x in Y_test:
    if x==0:
        Y_test_sparse.append([1,0])
    else:
        Y_test_sparse.append([0,1])
Y_test_sparse=np.array(Y_test_sparse)

In [61]:
from sklearn.metrics import roc_auc_score
print("Bidir AUROC",roc_auc_score(Y_test_sparse,pred_test_bidir_logits))
print("Bidir AUPRC",sklearn.metrics.average_precision_score(Y_test_sparse,pred_test_bidir_logits))

Bidir AUROC 0.904076875399143
Bidir AUPRC 0.8668302012199416


In [63]:
RNN_OUTPUT_UNITS = [64, 128]
import keras
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from sklearn.metrics import f1_score, accuracy_score
def get_model():
    inp = Input(shape=(187, 1))
    current_layer = keras.layers.Masking(mask_value=0., input_shape=(187, 1), name='Masked')(inp)
    for i, size in enumerate(RNN_OUTPUT_UNITS):
        notLast = i + 1 < len(RNN_OUTPUT_UNITS)
        current_layer = keras.layers.SimpleRNN(size, return_sequences=notLast, dropout=0.2, name = 'rnn' + str(i+1))(current_layer)

    current_layer = keras.layers.Dense(64, name='Dense1', activation='relu')(current_layer)
    current_layer = keras.layers.Dense(16, name='Dense2', activation='relu')(current_layer)
    logits = keras.layers.Dense(2, name='Output', activation='softmax')(current_layer)
    model = models.Model(inputs=inp, outputs=logits)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model
model = get_model()
file_path = "rnn_simple_ptb.h5"
model.load_weights(file_path)
pred_test_rnn_logits = model.predict(X_test)
pred_test_rnn= np.argmax(pred_test_rnn_logits, axis=-1)

Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 187, 1)            0         
_________________________________________________________________
Masked (Masking)             (None, 187, 1)            0         
_________________________________________________________________
rnn1 (SimpleRNN)             (None, 187, 64)           4224      
_________________________________________________________________
rnn2 (SimpleRNN)             (None, 128)               24704     
_________________________________________________________________
Dense1 (Dense)               (None, 64)                8256      
_________________________________________________________________
Dense2 (Dense)               (None, 16)                1040      
_________________________________________________________________
Output (Dense)               (None, 2)                 34 

In [64]:
print("RNN AUROC",roc_auc_score(Y_test_sparse,pred_test_rnn_logits))
print("RNN AUPRC",sklearn.metrics.average_precision_score(Y_test_sparse,pred_test_rnn_logits))

RNN AUROC 0.5948397782322798
RNN AUPRC 0.5734974808766539


In [65]:
RNN_OUTPUT_UNITS = [32,64, 128]
import keras
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from sklearn.metrics import f1_score, accuracy_score
def get_model():
    inp = Input(shape=(187, 1))
    current_layer = keras.layers.Masking(mask_value=0., input_shape=(187, 1), name='Masked')(inp)
    for i, size in enumerate(RNN_OUTPUT_UNITS):
        notLast = i + 1 < len(RNN_OUTPUT_UNITS)
        current_layer = keras.layers.LSTM(size, return_sequences=notLast, dropout=0.2, name = 'lstm' + str(i+1))(current_layer)

    current_layer = keras.layers.Dense(64, name='Dense1', activation='relu')(current_layer)
    current_layer = keras.layers.Dense(16, name='Dense2', activation='relu')(current_layer)
    logits = keras.layers.Dense(2, name='Output', activation='softmax')(current_layer)
    model = models.Model(inputs=inp, outputs=logits)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model

model = get_model()
file_path = "lstm_simple_ptb.h5"
model.load_weights(file_path)
pred_test_lstm_logits= model.predict(X_test)
pred_test_lstm= np.argmax(pred_test_lstm_logits, axis=-1)

Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 187, 1)            0         
_________________________________________________________________
Masked (Masking)             (None, 187, 1)            0         
_________________________________________________________________
lstm1 (LSTM)                 (None, 187, 32)           4352      
_________________________________________________________________
lstm2 (LSTM)                 (None, 187, 64)           24832     
_________________________________________________________________
lstm3 (LSTM)                 (None, 128)               98816     
_________________________________________________________________
Dense1 (Dense)               (None, 64)                8256      
_________________________________________________________________
Dense2 (Dense)               (None, 16)                104

In [66]:
from sklearn.metrics import roc_auc_score
print("LSTM AUROC",roc_auc_score(Y_test_sparse,pred_test_lstm_logits))
print("LSTM AUPRC",sklearn.metrics.average_precision_score(Y_test_sparse,pred_test_lstm_logits))

LSTM AUROC 0.7568740818973982
LSTM AUPRC 0.6963438696063017


In [67]:
RNN_OUTPUT_UNITS = [64, 128]
import keras
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from sklearn.metrics import f1_score, accuracy_score
def get_model():
    inp = Input(shape=(187, 1))
    current_layer = keras.layers.Masking(mask_value=0., input_shape=(187, 1), name='Masked')(inp)
    for i, size in enumerate(RNN_OUTPUT_UNITS):
        notLast = i + 1 < len(RNN_OUTPUT_UNITS)
        current_layer = keras.layers.GRU(size, return_sequences=notLast, dropout=0.2, name = 'GRU' + str(i+1))(current_layer)

    current_layer = keras.layers.Dense(64, name='Dense1', activation='relu')(current_layer)
    current_layer = keras.layers.Dense(16, name='Dense2', activation='relu')(current_layer)
    logits = keras.layers.Dense(2, name='Output', activation='softmax')(current_layer)
    model = models.Model(inputs=inp, outputs=logits)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model

model = get_model()
file_path = "gru_simple_ptb.h5"
model.load_weights(file_path)
pred_test_gru_logits= model.predict(X_test)
pred_test_gru= np.argmax(pred_test_gru_logits, axis=-1)

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 187, 1)            0         
_________________________________________________________________
Masked (Masking)             (None, 187, 1)            0         
_________________________________________________________________
GRU1 (GRU)                   (None, 187, 64)           12672     
_________________________________________________________________
GRU2 (GRU)                   (None, 128)               74112     
_________________________________________________________________
Dense1 (Dense)               (None, 64)                8256      
_________________________________________________________________
Dense2 (Dense)               (None, 16)                1040      
_________________________________________________________________
Output (Dense)               (None, 2)                 34 

In [68]:
from sklearn.metrics import roc_auc_score
print("GRU AUROC",roc_auc_score(Y_test_sparse,pred_test_gru_logits))
print("GRU AUPRC",sklearn.metrics.average_precision_score(Y_test_sparse,pred_test_gru_logits))

GRU AUROC 0.8459945734182173
GRU AUPRC 0.8256475867656561


In [69]:
import keras
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from sklearn.metrics import f1_score, accuracy_score
def conv_unit(unit, input_layer):
    s = '_' + str(unit)
    layer = Convolution1D(name='Conv1' + s, filters=32, kernel_size=5, strides=1, padding='same', activation='relu')(input_layer)
    layer = Convolution1D(name='Conv2' + s, filters=32, kernel_size=5, strides=1, padding='same', activation=None)(layer )
    layer = keras.layers.Add(name='ResidualSum' + s)([layer, input_layer])
    layer = keras.layers.Activation("relu", name='Act' + s)(layer)
    layer = keras.layers.MaxPooling1D(name='MaxPool' + s, pool_size=5, strides=2)(layer)
    return layer

def get_model():
    inp = Input(shape=(187, 1))
    current_layer = Convolution1D(filters=32, kernel_size=5, strides=1)(inp)

    for i in range(5):
        current_layer = conv_unit(i + 1, current_layer)

    current_layer = keras.layers.Flatten()(current_layer)
    current_layer = keras.layers.Dense(64, name='FC1', activation='relu')(current_layer)
    current_layer = keras.layers.Dense(32, name='FC2', activation='relu')(current_layer)
    logits = keras.layers.Dense(2,activation=activations.softmax, name='Output')(current_layer)

    model = models.Model(inputs=inp, outputs=logits)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model
model = get_model()
file_path = "residual_cnn_ptb.h5"
model.load_weights(file_path)
pred_test_res_cnn_logits= model.predict(X_test)
pred_test_res_cnn= np.argmax(pred_test_res_cnn_logits, axis=-1)

Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           (None, 187, 1)       0                                            
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 183, 32)      192         input_18[0][0]                   
__________________________________________________________________________________________________
Conv1_1 (Conv1D)                (None, 183, 32)      5152        conv1d_4[0][0]                   
__________________________________________________________________________________________________
Conv2_1 (Conv1D)                (None, 183, 32)      5152        Conv1_1[0][0]                    
___________________________________________________________________________________________

In [70]:
from sklearn.metrics import roc_auc_score
print("Residual CNN",roc_auc_score(Y_test_sparse,pred_test_res_cnn_logits))
print("Residual AUPRC",sklearn.metrics.average_precision_score(Y_test_sparse,pred_test_res_cnn_logits))

Residual CNN 0.9970651589692082
Residual AUPRC 0.9973457021022538


In [46]:
all_votes=np.concatenate([np.reshape(pred_test_res_cnn,[2911,1]),np.reshape(pred_test_bidir,[2911,1]),np.reshape(pred_test_gru,[2911,1]),np.reshape(pred_test_lstm,[2911,1]),np.reshape(pred_test_rnn,[2911,1])],axis=1)

In [47]:
from scipy.stats import mode
modes=mode(all_votes,axis=1)

In [48]:
modes_only=np.array(np.squeeze(modes[0]))

In [49]:
f1 = f1_score(Y_test, modes_only,average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, modes_only)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.7004557922662094 
Test accuracy score : 0.8179319821367228 


In [50]:
true_logits_bidir=[]
for i in range(0,np.shape(pred_test_bidir)[0]):
    true_logits_bidir.append(pred_test_bidir_logits[i,Y_test[i]])
true_logits_bidir=np.array(true_logits_bidir)
true_logits_rnn=[]
for i in range(0,np.shape(pred_test_rnn)[0]):
    true_logits_rnn.append(pred_test_rnn_logits[i,Y_test[i]])
true_logits_rnn=np.array(true_logits_rnn)
true_logits_lstm=[]
for i in range(0,np.shape(pred_test_lstm)[0]):
    true_logits_lstm.append(pred_test_lstm_logits[i,Y_test[i]])
true_logits_lstm=np.array(true_logits_lstm)
true_logits_gru=[]
for i in range(0,np.shape(pred_test_gru)[0]):
    true_logits_gru.append(pred_test_gru_logits[i,Y_test[i]])
true_logits_gru=np.array(true_logits_gru)
true_logits_res_cnn=[]
for i in range(0,np.shape(pred_test_res_cnn)[0]):
    true_logits_res_cnn.append(pred_test_res_cnn_logits[i,Y_test[i]])
true_logits_res_cnn=np.array(true_logits_res_cnn)

In [51]:
full_logits=np.concatenate([np.reshape(true_logits_bidir,[2911,1]),
                           np.reshape(true_logits_rnn,[2911,1]),
                           np.reshape(true_logits_lstm,[2911,1]),
                           np.reshape(true_logits_gru,[2911,1]),
                           np.reshape(true_logits_res_cnn,[2911,1])],axis=1)

In [52]:
full_logits_sum=np.sum(full_logits,axis=1)/5

In [53]:
from sklearn.linear_model import LogisticRegression

In [54]:
clf = LogisticRegression(random_state=0).fit(full_logits,Y_test)

/Users/rheasukthanker/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [55]:
preds=clf.predict(full_logits)

In [56]:
f1 = f1_score(Y_test, preds,average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, preds)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.9786756664835309 
Test accuracy score : 0.983167296461697 
